In [33]:
from brayns import Client
from circuitexplorer import CircuitExplorer

In [34]:
brayns = Client('localhost:5000')
ce = CircuitExplorer(brayns)

In [ ]:
config = '/home/favreau/git/install/release/share/BBPTestData/circuitBuilding_1000neurons/BlueConfig'

In [ ]:
''' Single cell '''
model = ce.load_circuit(
    use_sdf=True,
    load_soma=True, load_axon=True, load_apical_dendrite=True, load_dendrite=True,
    circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
    morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_BY_SECTION_TYPE,
    density=1, path=config, targets=['mini50'], mitochondria_density=0.1)

In [ ]:
def set_materials(model_id, palette_name, opacity=1.0):
    
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    import seaborn as sns
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    palette = sns.color_palette(palette_name, nb_materials)
    
    for i in range(nb_materials):
        c=palette[i]
        opacities.append(opacity)
        refraction_indices.append(1.5)
        reflection_indices.append(0.0)
        specular_exponents.append(50)
        glossinesses.append(1)
        diffuse_colors.append([c[0],c[1],c[2]])
        specular_colors.append([c[0],c[1],c[2]])
        shading_modes.append(ce.SHADING_MODE_DIFFUSE)
        simulation_data_casts.append(False)
        emissions.append(0)
        user_parameters.append(1)
        clips.append(False)
        
    print(material_ids)

    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)

In [ ]:
set_materials(model['id'], 'Set2', 0.5)

# Rat Atlas

In [35]:
import glob
import os
nrrd_folder = '/home/favreau/python/data/waxholm/brain_regions/brain_region/'
mesh_files = glob.glob(nrrd_folder + '*.obj')
for mesh_file in mesh_files:
    region_id = int(os.path.basename(mesh_file).split('.')[0])
    if region_id > 20 and region_id<1000 and region_id not in [58, 41]:
        brayns.add_model(mesh_file)

In [59]:
def set_model_color(model_id, color, opacity=1.0):
    simulation_data_casts=list()
    opacities=list()
    refraction_indices=list()
    reflection_indices=list()
    shading_modes=list()
    diffuse_colors=list()
    specular_colors=list()
    specular_exponents=list()
    material_ids=list()
    glossinesses=list()
    emissions=list()
    clips=list()
    user_parameters=list()
    
    material_ids = ce.get_material_ids(model_id)['ids']
    nb_materials = len(material_ids)
    for i in range(nb_materials):
        
        if True: # noise
            opacities.append(0.5)
            refraction_indices.append(1.5)
            specular_exponents.append(50)
            glossinesses.append(0.5)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            user_parameters.append(0.001)
        else:
            opacities.append(0.75)
            refraction_indices.append(1.0)
            specular_exponents.append(5)
            glossinesses.append(1.0)
            shading_modes.append(ce.SHADING_MODE_ELECTRON)
            user_parameters.append(1.0)

        c=color
        reflection_indices.append(0.0)
        diffuse_colors.append([c[0],c[1],c[2]])
        specular_colors.append([c[0],c[1],c[2]])
        simulation_data_casts.append(False)
        emissions.append(0)
        clips.append(True)

    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        simulation_data_casts=simulation_data_casts,
        opacities=opacities, reflection_indices=reflection_indices,
        shading_modes=shading_modes, user_parameters=user_parameters,
        diffuse_colors=diffuse_colors, specular_colors=specular_colors,
        specular_exponents=specular_exponents, glossinesses=glossinesses,
        emissions=emissions, refraction_indices=refraction_indices)

In [61]:
import seaborn as sns
models = brayns.scene.models
nb_models = len(models)
palette = sns.color_palette('pink', nb_models)
for i in range(nb_models):
    model = models[i]
    model_id = model['id']
    ce.set_material_extra_attributes(model_id)
    set_model_color(model_id, palette[i])